In [1]:
import torch.backends.mps

from decoder import AttnDecoderRNN
from encoder import EncoderRNN
from evaluation_functions import inference, evaluate_bertscore
from dataloader import Lang, prepare_data
from io import open
import os
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f'using device: {device}')

using device: mps


In [2]:
loaded_data = 1000
validation_size = 300
train_size = 1
max_length = 30  # max length of 30 retains around 1/3 of the data; 20 => 1/8

input_lang, output_lang, english_sequences, german_sequences, validation_english, validation_german = \
    prepare_data('data/train.en', 'data/train.de', max_length, train_size, validation_size,
                 loaded_data=loaded_data, device=device)

Filtered to 346 sentence pairs
Train sequences: 1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00, 234.42it/s]


In [3]:
models_dir = "models_gru"
model_name = "5e-5_lr_1_layer_100_hidden_2"
encoder = torch.load(os.path.join(models_dir, model_name, "encoder.pt"), map_location=device)
decoder = torch.load(os.path.join(models_dir, model_name, "decoder.pt"), map_location=device)

In [6]:
bert_score = evaluate_bertscore(encoder, decoder, validation_english, validation_german, input_lang, output_lang, 30, "facebook/bart-large-mnli", device)
bert_score

8.291608810424805


[0.389435350894928,
 0.356946736574173,
 0.36033084988594055,
 0.3768613338470459,
 0.3851816654205322,
 0.3716244399547577,
 0.36954617500305176,
 0.37553325295448303,
 0.36251100897789,
 0.3711259365081787,
 0.4110725224018097,
 0.4324071705341339,
 0.38086700439453125,
 0.3541617691516876,
 0.41357097029685974,
 0.3884272575378418,
 0.380095899105072,
 0.39330539107322693,
 0.34811633825302124,
 0.39452847838401794,
 0.3903738856315613,
 0.4500541388988495,
 0.4144149720668793,
 0.36120980978012085,
 0.39022788405418396,
 0.3812312185764313,
 0.3748236298561096,
 0.40792056918144226,
 0.4093226492404938,
 0.37486594915390015,
 0.3817298710346222,
 0.4157201051712036,
 0.3853645324707031,
 0.3992069959640503,
 0.3734957277774811,
 0.3587798774242401,
 0.3531423807144165,
 0.3806183934211731,
 0.39056965708732605,
 0.3757345378398895,
 0.40238258242607117,
 0.3632217049598694,
 0.38252297043800354,
 0.3610265552997589,
 0.4040934145450592,
 0.4743151068687439,
 0.4495171904563904,
 0.

In [7]:
bert_score = evaluate_bertscore(encoder, decoder, validation_english, validation_german, input_lang, output_lang, 30, "microsoft/deberta-xlarge-mnli", device)
bert_score

18.191924810409546


[0.4209511876106262,
 0.422923743724823,
 0.4632260799407959,
 0.34269681572914124,
 0.4507923126220703,
 0.3699100911617279,
 0.39163830876350403,
 0.4027002155780792,
 0.35307982563972473,
 0.4254568815231323,
 0.4391911029815674,
 0.4642578959465027,
 0.38709384202957153,
 0.4413072466850281,
 0.42415034770965576,
 0.38990527391433716,
 0.44043976068496704,
 0.4513382017612457,
 0.45362773537635803,
 0.4717690348625183,
 0.43379971385002136,
 0.4346602261066437,
 0.44217076897621155,
 0.39455941319465637,
 0.44540920853614807,
 0.4708751440048218,
 0.44468367099761963,
 0.39874961972236633,
 0.3694831430912018,
 0.3689633011817932,
 0.46965292096138,
 0.5043194890022278,
 0.46716374158859253,
 0.4638514518737793,
 0.4258023798465729,
 0.41160547733306885,
 0.3611133098602295,
 0.44868266582489014,
 0.41786783933639526,
 0.4218587577342987,
 0.45299363136291504,
 0.4682697057723999,
 0.3771536350250244,
 0.3770250380039215,
 0.4611715078353882,
 0.4504491984844208,
 0.463349401950836

In [8]:
len(validation_english)

300

In [27]:
from evaluate import load
from time import time
bertscore = load("bertscore")

In [33]:
t = time()
predictions = ["Das Hotel ist zentral", "general kenobi"][:1]
references = ["Das Hotel ist zentral", "hello there"][:1]
bertscore.compute(predictions=predictions, references=references, model_type="facebook/bart-large-mnli", lang="de", device="cpu")
print(time() - t)

0.07848000526428223


In [34]:
t = time()
predictions = ["Das Hotel ist zentral", "general kenobi"][:1]
references = ["Das Hotel ist zentral", "hello there"][:1]
bertscore.compute(predictions=predictions, references=references, model_type="facebook/bart-large-mnli", lang="de", device="mps")
print(time() - t)

0.0687861442565918
